## Final Project Submission

Please fill out: 
* Student name: Lynette Wangari
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Asha Deen
* Blog post URL:


# MICROSOFT MOVIE STUDIO EXPLORATION AND DATA ANALYSIS

### 1) Introduction

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies.  

Our task is to analyze the current movie gentres at the Box office and their performance, considering bvarious factors that affect movie success.

This analysis will  provide insights to help Microsoft make informed decisions on which types of films to focus on. To start, we'll gather and analyze relevant data and clarify business objectives. This will establish a solid foundation and direction for the project.

### 2) Problem Statement
Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. 

You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

### 3) Data Sets Used
The movie datasets are from the following websites:
1. https://www.boxofficemojo.com/
2. https://www.imdb.com/
3. https://www.rottentomatoes.com/
4. https://www.themoviedb.org/
5. https://www.the-numbers.com/


### 4) Project Structure

1. Business Understanding
2. Data Understanding
3. Data Cleaning and Preparation
4. Exploratory Data Analysis
5. Visualization: Insights Derived from the Values
6. Recommendations: Key findings

#### 2. Data Understanding
 Our data is stored in a folder called zippedData. So we will first import the necessary packages then we'll explore the data by loading the datasets. 

In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import seaborn as sns
%matplotlib inline

In [11]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('zippedData/im.db')

# Create a cursor object
cur = conn.cursor()

# Execute the SQL query to fetch the list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cur.fetchall()

# Print the list of tables
print(tables)

# Close the connection
conn.close()

[('movie_basics',), ('directors',), ('known_for',), ('movie_akas',), ('movie_ratings',), ('persons',), ('principals',), ('writers',)]


In [12]:
#import data from db
bom_movie = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
reviews = pd.read_csv('./zippedData/rt.reviews.tsv.gz',sep="\t", encoding = 'unicode_escape')
tmdb = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
budget = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')


In [20]:
import sqlite3
import pandas as pd

# Path to the SQLite database file
db_file_path = r'C:\Users\User\Documents\GitHub\dsc-phase-1-project-v2-4\zippedData\im.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Read data from the movie_basics table
movie_basics = pd.read_sql("""
    SELECT *
    FROM movie_basics
""", conn)

# Display the first few rows of the DataFrame
movie_basics.head()



,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [21]:
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
